In [4]:
%pip install openmeteo-requests retry-requests requests-cache numpy pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [requests-cache]m [cattrs]
Note: you may need to restart the kernel to use updated packages.


In [7]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# -------- 1. Client Open-Meteo avec cache + retry --------
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"

# -------- 2. Coordonnées des départements d'Île-de-France --------
idf_departements = [
    {"code": "75", "name": "Paris",            "lat": 48.8566, "lon": 2.3522},
    {"code": "92", "name": "Hauts-de-Seine",   "lat": 48.85,   "lon": 2.19},
    {"code": "93", "name": "Seine-Saint-Denis","lat": 48.91,   "lon": 2.31},
    {"code": "94", "name": "Val-de-Marne",     "lat": 48.78,   "lon": 2.49},
    {"code": "91", "name": "Essonne",          "lat": 48.46,   "lon": 2.16},
    {"code": "78", "name": "Yvelines",         "lat": 48.76,   "lon": 1.28},
    {"code": "95", "name": "Val-d'Oise",       "lat": 49.07,   "lon": 1.82},
    {"code": "77", "name": "Seine-et-Marne",   "lat": 48.65,   "lon": 2.96},
]

# -------- 3. Liste des années à récupérer --------
# Ici 2018 → 2024 inclus. Tu peux adapter facilement.
years = range(2018, 2025)

for year in years:
    print(f"\n===== Année {year} =====")
    all_dfs = []

    for dep in idf_departements:
        print(f"Récupération pour {dep['name']} ({dep['code']})...")

        params = {
            "latitude":  dep["lat"],
            "longitude": dep["lon"],
            "start_date": f"{year}-01-01",
            "end_date":   f"{year}-12-31",
            "hourly": [
                "temperature_2m",
                "wind_speed_10m",
                "cloud_cover",
                "direct_radiation",
                "diffuse_radiation"
            ],
            "timezone": "UTC"
        }

        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        hourly = response.Hourly()

        # Extraction des variables météo
        temp    = hourly.Variables(0).ValuesAsNumpy()
        wind    = hourly.Variables(1).ValuesAsNumpy()
        cloud   = hourly.Variables(2).ValuesAsNumpy()
        direct  = hourly.Variables(3).ValuesAsNumpy()
        diffuse = hourly.Variables(4).ValuesAsNumpy()

        # Index temporel (heures de l'année)
        dates = pd.date_range(
            start=pd.to_datetime(hourly.Time(),    unit="s", utc=True),
            end  =pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq =pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        df_dep = pd.DataFrame({
            "date": dates,
            "temperature_2m": temp,
            "wind_speed_10m": wind,
            "cloud_cover": cloud,
            "direct_radiation": direct,
            "diffuse_radiation": diffuse,
            "dep_code": dep["code"],
            "dep_name": dep["name"],
            "lat": dep["lat"],
            "lon": dep["lon"],
        })

        all_dfs.append(df_dep)

    # Concat pour tous les départements de l'année
    df_year = pd.concat(all_dfs, ignore_index=True)

    # -------- 4. Sauvegarde : un CSV par année --------
    output_file = f"weather_idf_departements_{year}.csv"
    df_year.to_csv(output_file, index=False)

    print(f"✅ Sauvegardé dans '{output_file}' ({df_year.shape[0]} lignes, {df_year.shape[1]} colonnes)")



===== Année 2018 =====
Récupération pour Paris (75)...
Récupération pour Hauts-de-Seine (92)...
Récupération pour Seine-Saint-Denis (93)...
Récupération pour Val-de-Marne (94)...
Récupération pour Essonne (91)...
Récupération pour Yvelines (78)...
Récupération pour Val-d'Oise (95)...
Récupération pour Seine-et-Marne (77)...
✅ Sauvegardé dans 'weather_idf_departements_2018.csv' (70080 lignes, 10 colonnes)

===== Année 2019 =====
Récupération pour Paris (75)...
Récupération pour Hauts-de-Seine (92)...
Récupération pour Seine-Saint-Denis (93)...
Récupération pour Val-de-Marne (94)...
Récupération pour Essonne (91)...
Récupération pour Yvelines (78)...
Récupération pour Val-d'Oise (95)...
Récupération pour Seine-et-Marne (77)...
✅ Sauvegardé dans 'weather_idf_departements_2019.csv' (70080 lignes, 10 colonnes)

===== Année 2020 =====
Récupération pour Paris (75)...
Récupération pour Hauts-de-Seine (92)...
Récupération pour Seine-Saint-Denis (93)...
Récupération pour Val-de-Marne (94)...
Ré

OpenMeteoRequestsError: failed to request 'https://archive-api.open-meteo.com/v1/archive': {'error': True, 'reason': 'Minutely API request limit exceeded. Please try again in one minute.'}

In [8]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# -------- 1. Client Open-Meteo avec cache + retry --------
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"

# -------- 2. Coordonnées des départements d'Île-de-France --------
idf_departements = [
    {"code": "75", "name": "Paris",            "lat": 48.8566, "lon": 2.3522},
    {"code": "92", "name": "Hauts-de-Seine",   "lat": 48.85,   "lon": 2.19},
    {"code": "93", "name": "Seine-Saint-Denis","lat": 48.91,   "lon": 2.31},
    {"code": "94", "name": "Val-de-Marne",     "lat": 48.78,   "lon": 2.49},
    {"code": "91", "name": "Essonne",          "lat": 48.46,   "lon": 2.16},
    {"code": "78", "name": "Yvelines",         "lat": 48.76,   "lon": 1.28},
    {"code": "95", "name": "Val-d'Oise",       "lat": 49.07,   "lon": 1.82},
    {"code": "77", "name": "Seine-et-Marne",   "lat": 48.65,   "lon": 2.96},
]

# -------- 3. Liste des années à récupérer --------
# Ici 2018 → 2024 inclus. Tu peux adapter facilement.
years = range(2022, 2025)

for year in years:
    print(f"\n===== Année {year} =====")
    all_dfs = []

    for dep in idf_departements:
        print(f"Récupération pour {dep['name']} ({dep['code']})...")

        params = {
            "latitude":  dep["lat"],
            "longitude": dep["lon"],
            "start_date": f"{year}-01-01",
            "end_date":   f"{year}-12-31",
            "hourly": [
                "temperature_2m",
                "wind_speed_10m",
                "cloud_cover",
                "direct_radiation",
                "diffuse_radiation"
            ],
            "timezone": "UTC"
        }

        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        hourly = response.Hourly()

        # Extraction des variables météo
        temp    = hourly.Variables(0).ValuesAsNumpy()
        wind    = hourly.Variables(1).ValuesAsNumpy()
        cloud   = hourly.Variables(2).ValuesAsNumpy()
        direct  = hourly.Variables(3).ValuesAsNumpy()
        diffuse = hourly.Variables(4).ValuesAsNumpy()

        # Index temporel (heures de l'année)
        dates = pd.date_range(
            start=pd.to_datetime(hourly.Time(),    unit="s", utc=True),
            end  =pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq =pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        df_dep = pd.DataFrame({
            "date": dates,
            "temperature_2m": temp,
            "wind_speed_10m": wind,
            "cloud_cover": cloud,
            "direct_radiation": direct,
            "diffuse_radiation": diffuse,
            "dep_code": dep["code"],
            "dep_name": dep["name"],
            "lat": dep["lat"],
            "lon": dep["lon"],
        })

        all_dfs.append(df_dep)

    # Concat pour tous les départements de l'année
    df_year = pd.concat(all_dfs, ignore_index=True)

    # -------- 4. Sauvegarde : un CSV par année --------
    output_file = f"weather_idf_departements_{year}.csv"
    df_year.to_csv(output_file, index=False)

    print(f"✅ Sauvegardé dans '{output_file}' ({df_year.shape[0]} lignes, {df_year.shape[1]} colonnes)")



===== Année 2022 =====
Récupération pour Paris (75)...
Récupération pour Hauts-de-Seine (92)...
Récupération pour Seine-Saint-Denis (93)...
Récupération pour Val-de-Marne (94)...
Récupération pour Essonne (91)...
Récupération pour Yvelines (78)...
Récupération pour Val-d'Oise (95)...
Récupération pour Seine-et-Marne (77)...
✅ Sauvegardé dans 'weather_idf_departements_2022.csv' (70080 lignes, 10 colonnes)

===== Année 2023 =====
Récupération pour Paris (75)...
Récupération pour Hauts-de-Seine (92)...
Récupération pour Seine-Saint-Denis (93)...
Récupération pour Val-de-Marne (94)...
Récupération pour Essonne (91)...
Récupération pour Yvelines (78)...
Récupération pour Val-d'Oise (95)...
Récupération pour Seine-et-Marne (77)...
✅ Sauvegardé dans 'weather_idf_departements_2023.csv' (70080 lignes, 10 colonnes)

===== Année 2024 =====
Récupération pour Paris (75)...
Récupération pour Hauts-de-Seine (92)...
Récupération pour Seine-Saint-Denis (93)...
Récupération pour Val-de-Marne (94)...
Ré